In [ ]:
%%capture
import os, glob, sys, shutil
import importlib.util
import numpy as np

packages=['pydicom==2.4.4','spec2nii==0.7.3']
for package in packages:
    if importlib.util.find_spec(package.split('==')[0]) is None:
        !{sys.executable} -m pip install {package}
import pydicom

In [3]:
#collect dicom spectras, sort them, convert to nifti/json and rename to bids
def spec2bids(indir,outdir,subject,session):
    spec_tmp_folder=os.path.join(outdir,'spectmp')
    act_folder=os.path.join(spec_tmp_folder,'act')
    ref_folder=os.path.join(spec_tmp_folder,'ref')
    if not os.path.exists(spec_tmp_folder):
        os.makedirs(act_folder)
        os.makedirs(ref_folder)

    #collect and sort dicoms
    files=glob.glob(os.path.join(indir,'**','XX*'),recursive=True)
    print(files)
    ds=pydicom.dcmread(files[0])
    if ds.file_meta.ImplementationVersionName.endswith('54.1'):
        print('old')
        specorder=np.arange(1,321)
    elif ds.file_meta.ImplementationVersionName.endswith('57.0'):
        print('new')
        specorder=np.reshape(np.vstack((np.arange(1,161),np.arange(161,321))),-1,order='F')
    else:
        raise Exception("Unknown Philips version") 
    for f in files:
        ds=pydicom.dcmread(f)
        if (0x2005,0x1313) in ds:
            if ds[0x2005,0x1313].value<=320:
                shutil.copy(f,os.path.join(act_folder,f"{specorder[ds[0x2005,0x1313].value-1]:03d}"+'.dcm'))
            else:
                shutil.copy(f,ref_folder)

    #dcm2nii conversion
    !spec2nii philips_dcm -j -f 'act' -o {spec_tmp_folder} {act_folder}
    !spec2nii philips_dcm -j -f 'ref' -o {spec_tmp_folder} {ref_folder}

    #bids renaming
    voxel='dACC'
    specname='sub-'+subject+'_ses-'+session+'_acq-megapress_nuc-1H_voi-'+voxel
    svsname=specname+'_svs'
    refname=specname+'_ref'
    os.rename(os.path.join(spec_tmp_folder,'act.nii.gz'),os.path.join(outdir,svsname+'.nii.gz'))
    os.rename(os.path.join(spec_tmp_folder,'act.json'),os.path.join(outdir,svsname+'.json'))
    os.rename(os.path.join(spec_tmp_folder,'ref.nii.gz'),os.path.join(outdir,refname+'.nii.gz'))
    os.rename(os.path.join(spec_tmp_folder,'ref.json'),os.path.join(outdir,refname+'.json'))

    !rm -r {spec_tmp_folder}